In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import collections

In [8]:
preverbsDF = pd.read_csv('../data/WiyotPreverbsText.tsv',sep='\t')
preverbsDF.head()

,Wiyot,English
0,bał/wał/wáł,relating.to
1,be/bé/béh,REP
2,bi/mi/mí,soon
3,bime/mime/bige/bihge/mige,some
4,bo/bó/wo/wó/ba/wa/boh,go.to


In [9]:
preverbs = list(set(preverbsDF['English']))
len(preverbs)

81

In [13]:
kroeber = pd.read_excel(f'../data/Kroeber texts (1).xlsx',None)
reichard = pd.read_excel(f'../data/Reichard texts (1).xlsx',None)
teeter = pd.read_excel(f'../data/Teeter texts (1).xlsx',None)

/Users/leonjdaeim/opt/miniconda3/envs/EDA/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [14]:
def processLine(x):
    try:
        x = '\t'.join(x.dropna()).split('\t')
        return x
    except:
        return None
    

## Remove empty lines
import re
matchEndPeriod = re.compile(r'\.$')
getlowercase = lambda x: x if x.isupper() else x.lower()    
_isna = lambda x: x.isna()

def getWordPairs(phrase):
    wordpairlist = []
    for i in range(0,len(phrase)-1):
        wordpairlist.append('|'.join([phrase[i],phrase[i+1]]))
    return wordpairlist

def countFrequencyWordPairs(listofwordpairs):
    try:
        return collections.Counter(listofwordpairs[:])
    except:
        return None

In [16]:
def processStory(author,story):
    data = author[story]
    data = data[~data.apply(_isna,0).all(1)]
    #####
    wiyotList = []
    originalEnglishList = []
    processedEnglishList = []
    preverbsEnglishList = []
    pairsEnglishList = []
    ######
    for i in range(0,len(data),2):   
        ##########################################
        wiyotPhrase = data.iloc[i]
        englishPhrase = data.iloc[i+1]
        wiyotPhrase = processLine(wiyotPhrase)    
        ### Do some extra processing to the english phrase
        englishPhrase = processLine(englishPhrase)
        #englishPhrase = [getlowercase(x) for x in englishPhrahaha se]
        ### Remove EMP = empty, remove periods at the end of gloss
        englishPhraseRedux = [getlowercase(matchEndPeriod.sub('',x)) for x in englishPhrase if x !='EMP'] #Remove EMP for empty 
        englishPreverbs = [_ for _ in englishPhraseRedux if _ in preverbs] 

        ### Get word pairs
        wps = getWordPairs(englishPhraseRedux)

        wiyotList.append(wiyotPhrase)
        originalEnglishList.append(englishPhrase)
        processedEnglishList.append(englishPhraseRedux)
        preverbsEnglishList.append(englishPreverbs)
        pairsEnglishList.append(wps)
        
    outDict = {"wiyot":wiyotList,
               "english_original":originalEnglishList,
              "english_clean":processedEnglishList,
              "english_preverbs":preverbsEnglishList,
              'wordPairs':pairsEnglishList}
    return outDict

def getPreverbs(dataset):
    try:
        #dPrev = {k:v['english_preverbs'] for k,v in dataset.items()}
        #return pd.DataFrame.from_dict(dPrev,orient='index')
        author_preverbs = [dataset[k]['english_preverbs'] for k in dataset.keys()]
        author_preverbs = sum(sum(author_preverbs,[]),[])
        return author_preverbs
    except:
        return []

In [17]:
kroeber_processed = {story:processStory(kroeber,story) for story in kroeber.keys()}
reichard_processed = {story:processStory(reichard,story) for story in reichard.keys()}
teeter_processed = {story:processStory(teeter,story) for story in teeter.keys()}

### Get preverb frequency

In [19]:
reichard_preverbs = getPreverbs(reichard_processed)
teeter_preverbs = getPreverbs(teeter_processed)
kroeber_preverbs = getPreverbs(kroeber_processed)

In [20]:
allpreverbs = kroeber_preverbs + reichard_preverbs + teeter_preverbs

In [21]:
import collections

In [22]:
allpreverbs_frequencies = collections.Counter(allpreverbs)
allpreverbs_frequencies = pd.DataFrame.from_dict(allpreverbs_frequencies,orient='index',columns=['frequency'])
allpreverbs_frequencies = allpreverbs_frequencies.sort_values('frequency',ascending=False).reset_index().rename(columns={'index':'preverb'})
allpreverbs_frequencies.head()

,preverb,frequency
0,PROG,1629
1,PST,1333
2,INCH,835
3,ITER,783
4,3NEG,344


In [33]:
allpreverbs_frequencies.to_csv('../results/allpreverbs_frequencies.txt',sep='\t',index=None)
#allpreverbs_frequencies.to_clipboard(index=None)

In [25]:
def getPreverbsPairs(dataset):
    try:
        author_wordpairs = [[getWordPairs(_) for _ in dataset[k]['english_preverbs'] if len(_) >= 2] for k in dataset.keys()]
        author_wordpairs = sum(author_wordpairs,[])
        return author_wordpairs
    except:
        return []

In [27]:
kroeber_preverbPairs = getPreverbsPairs(kroeber_processed)
teeter_preverbPairs = getPreverbsPairs(teeter_processed)
reichard_preverbPairs = getPreverbsPairs(reichard_processed)

In [28]:
allPreverbPairs = sum(kroeber_preverbPairs + teeter_preverbPairs + reichard_preverbPairs,[])

In [29]:
allpreverbs_wordPair_frequencies = collections.Counter(allPreverbPairs)
allpreverbs_wordPair_frequencies = pd.DataFrame.from_dict(allpreverbs_wordPair_frequencies,orient='index',columns=['frequency'])
allpreverbs_wordPair_frequencies = allpreverbs_wordPair_frequencies.sort_values('frequency',ascending=False).reset_index().rename(columns={'index':'preverb_pair'})
allpreverbs_wordPair_frequencies.insert(0,'word_1',allpreverbs_wordPair_frequencies['preverb_pair'].map(lambda x: x.split('|')[0]))
allpreverbs_wordPair_frequencies.insert(1,'word_2',allpreverbs_wordPair_frequencies['preverb_pair'].map(lambda x: x.split('|')[1]))
allpreverbs_wordPair_frequencies.head()
allpreverbs_wordPair_frequencies.to_csv('../results/allpreverbs_pairs_frequencies.txt',sep='\t',index=None)

In [30]:
allpreverbs_wordPair_frequencies.to_clipboard(index=None) #Copy to shared Google sheet
### https://docs.google.com/spreadsheets/d/1nGYuJ1dr3s8la8gHm9hywFS3CJgd1_DfLYnOBWrAbfQ/edit#gid=1161947669

In [31]:
len(allpreverbs_wordPair_frequencies)

702

In [32]:
allpreverbs_wordPair_frequencies.tail()

,word_1,word_2,preverb_pair,frequency
697,PROB,then.2,PROB|then.2,1
698,PL,thoroughly,PL|thoroughly,1
699,3NEG,then,3NEG|then,1
700,first,all,first|all,1
701,ITER,thoroughly.2,ITER|thoroughly.2,1


#### Preverb hierarchy

In [52]:
preverbhierarchy = pd.read_csv('../data/preverbhierarchy.tsv',sep='\t')
preverbhierarchy.head()

,preverb,Class,Level
0,PST,I,1
1,already,I,1
2,EMPH,I,1
3,PROG,I,1
4,then,I,1


In [132]:
preverb2level = dict(zip(preverbhierarchy['preverb'],preverbhierarchy['Level']))
preverb2class = dict(zip(preverbhierarchy['preverb'],preverbhierarchy['Class']))

In [39]:
def getPreverbs(dataset):
    try:
        #dPrev = {k:v['english_preverbs'] for k,v in dataset.items()}
        #return pd.DataFrame.from_dict(dPrev,orient='index')
        author_preverbs = [dataset[k]['english_preverbs'] for k in dataset.keys()]
        #author_preverbs = sum(sum(author_preverbs,[]),[])
        return author_preverbs
    except:
        return []

In [120]:
checkpos = lambda x,y: x <= y
### checkpos(1,2),checkpos(5,2) #test
def checkSequenceClass(classseq):
    if len(classseq) > 1:
        res = all([checkpos(classseq[i-1],classseq[i]) for i in range(1,len(classseq))])
    else:
        res = None
    return res


In [76]:
reichard_processed['Sample Text'].keys()

dict_keys(['wiyot', 'english_original', 'english_clean', 'english_preverbs', 'wordPairs'])

### PoC for sample text

In [150]:
reichard_processed.keys()

dict_keys(['Sample Text', '1. Nettle Medicine', "2. Wolf's Home", '3. Spring Maiden', '4. Curlew', '5. North Wind', '6. Killer Whale', '7. Above-Old-Man', '8. Southwest-Young-Man Looks...', '9. Flute', '10. Owl and Coyote Smoke', '11. The Man Who Became a Woman', '12. Southwest-Young-Man Marries', '13. He-Who-Was-Dug-Up', '14. The Brother Who Became A...', '15. Slug', '16. Panther and Coyote', '17. Owl', '18. Aldebaran', '19. The Flood', "20. Eagle's Children Turn to...", '21. Coyote Gets Married', '22. Northwest-Young-Man', '23. Twine-Eater', '24. Frog and Sand Cricket', '25. The Sky Falls', '26. Coyote and Steelhead', '27. Morning Star', '28. Cormorant', '29. Wildcat Gets Caught', '30. Southeast-Old-Man', '31. Sleepy-Head', '32. Coyote and Panther (2)', '33. Coyote and the Bears', '34. Coyote and the Brodiaea...', '35. Sea Lion and Grizzly Bear', '36. Coyote Returns from the Sky', '37. Southwest-Young-Man Love...', "38. Wood's Devil", '39. Túthabił', '40. Big Lagoon', "41. Children's

In [184]:
def checkPreverbHierachy(author_processed):
    author = dict()
    for story in author_processed.keys(): 
        storyXwiyotOriginal = author_processed[story]['wiyot']
        storyXenglishOriginal = author_processed[story]['english_original']
        storyXpreverb = author_processed[story]['english_preverbs']
        results = []
        for i in range(len(storyXpreverb)):
            tmp = storyXpreverb[i]
            tmp_sublevel = [preverb2level[_] for _ in tmp if preverb2level.get(_) is not None]
            tmp_subclass = [preverb2class[_] if preverb2level.get(_) is not None else 0 for _ in tmp ]

            hierearchyresults = checkSequenceClass(tmp_sublevel)
            results.append([storyXwiyotOriginal[i],storyXenglishOriginal[i],hierearchyresults,tmp_subclass])
        resultsDF = pd.DataFrame(results,columns=['Wiyot','English','Hierarchy','Levels'])

        resultsDF['story'] = [story] * len(resultsDF)
        author[story] = resultsDF
    allstories = pd.concat(author.values())
    return allstories

In [191]:
reichard_classseq = checkPreverbHierachy(reichard_processed)
reichard_classseq.groupby('Hierarchy').size().reset_index().rename(columns={0:'n'})

,Hierarchy,n
0,False,317
1,True,624


In [189]:
teeter_classseq = checkPreverbHierachy(teeter_processed)
teeter_classseq.groupby('Hierarchy').size().reset_index().rename(columns={0:'n'})

,Hierarchy,0
0,False,375
1,True,708


In [192]:
kroeber_classseq = checkPreverbHierachy(kroeber_processed)
kroeber_classseq.groupby('Hierarchy').size().reset_index().rename(columns={0:'n'})

,Hierarchy,n
0,False,5
1,True,17


In [196]:
reichard_classseq['author'] = 'reichard'
kroeber_classseq['author'] = 'kroeber'
teeter_classseq['author'] = 'teeter'

In [202]:
allauthors = pd.concat([reichard_classseq,kroeber_classseq,teeter_classseq])
#allauthors.to_clipboard()

In [201]:
allauthors.groupby(['author','Hierarchy']).size().reset_index().rename(columns={0:'n'})

,author,Hierarchy,n
0,kroeber,False,5
1,kroeber,True,17
2,reichard,False,317
3,reichard,True,624
4,teeter,False,375
5,teeter,True,708


In [203]:
allauthors.groupby(['Hierarchy']).size().reset_index().rename(columns={0:'n'})

,Hierarchy,n
0,False,697
1,True,1349


In [204]:
1349, 1349+697

(1349, 2046)